In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller

In [ ]:
# Load dataset
df = pd.read_csv("stock_price.csv", parse_dates=["Date"], index_col="Date")

# Menampilkan 5 data pertama
print(df.head())

In [ ]:
def adf_test(series):
    result = adfuller(series)
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")
    print("Stationary" if result[1] < 0.05 else "Non-Stationary")

# Uji stasioneritas untuk setiap variabel
print("Stock_A:")
adf_test(df["Stock_A"])
print("\nStock_B:")
adf_test(df["Stock_B"])

In [ ]:
Jika data tidak stasioner (p-value > 0.05), kita ambil first difference:

df_diff = df.diff().dropna()

In [ ]:
# Menentukan lag optimal
model = VAR(df_diff)
lag_selection = model.select_order(maxlags=10)
print(lag_selection.summary())

In [ ]:
var_model = VAR(df_diff)
var_result = var_model.fit(lag_selection.aic)
print(var_result.summary())

In [ ]:
Prediksi 10 periode ke depan menggunakan hasil model:

n_forecast = 10
forecast_input = df_diff.values[-lag_selection.aic:]  # Data terakhir sesuai jumlah lag
forecast = var_result.forecast(forecast_input, steps=n_forecast)

# Membuat dataframe hasil prediksi
forecast_index = pd.date_range(start=df.index[-1], periods=n_forecast+1, freq="D")[1:]
forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=df.columns)

# Mengembalikan data ke skala asli (cumulative sum)
forecast_df = forecast_df.cumsum() + df.iloc[-1]

In [ ]:
# Visualisasi hasil prediksi
plt.figure(figsize=(10, 5))
plt.plot(df.index, df["Stock_A"], label="Actual Stock_A")
plt.plot(forecast_df.index, forecast_df["Stock_A"], label="Forecast Stock_A", linestyle="dashed")
plt.legend()
plt.show()